In [1]:
from pandas import DataFrame, Grouper
import pandas as pd
import numpy as np
from datetime import datetime
import sys

from IPython.display import HTML


In [2]:
# import our own stuff
sys.path.append("../python/src")

from tools import read_water_data


In [3]:
# set some options
pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 600)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

datadir = "../data/"
cachedir = "../data/cache" 

read all data provided by the oecd (check README.md for download locations of csv-files)

In [4]:
oecddf = read_water_data(setname="fullset",datadir=datadir,cachedir=cachedir)

Reading Datafrom cached file: ../data/cache/fullset.p


show a sample of the data

In [5]:
oecddf.sample()

,Year,DonorCode,DonorName,AgencyCode,AgencyName,CrsID,ProjectNumber,InitialReport,RecipientCode,RecipientName,RegionCode,RegionName,IncomegroupCode,IncomegroupName,FlowCode,FlowName,Bi_Multi,Category,Finance_t,Aid_t,USD_Commitment,USD_Disbursement,USD_Received,USD_Commitment_Defl,USD_Disbursement_Defl,USD_Received_Defl,USD_Adjustment,USD_Adjustment_Defl,USD_AmountUntied,USD_AmountPartialTied,USD_AmountTied,USD_AmountUntied_Defl,USD_AmountPartialTied_Defl,USD_Amounttied_Defl,USD_IRTC,USD_Expert_Commitment,USD_Expert_Extended,USD_Export_Credit,CurrencyCode,Commitment_National,Disbursement_National,GrantEquiv,USD_GrantEquiv,ShortDescription,ProjectTitle,PurposeCode,PurposeName,SectorCode,SectorName,ChannelCode,ChannelName,ChannelReportedName,ParentChannelCode,Geography,ExpectedStartDate,CompletionDate,LongDescription,SDGfocus,Gender,Environment,PDGG,Trade,RMNCH,DRR,Nutrition,Disability,FTC,PBA,InvestmentProject,AssocFinance,Biodiversity,ClimateMitigation,ClimateAdaptation,Desertification,CommitmentDate,TypeRepayment,NumberRepayment,Interest1,Interest2,Repaydate1,Repaydate2,USD_Interest,USD_Outstanding,USD_Arrears_Principal,USD_Arrears_Interest,BudgetIdent,CapitalExpend,PSIflag,PSIAddType,PSIAddAssess,PSIAddDevObj
161251,2017,11,Switzerland,4,Swiss Agency for Development and Co-operation,2017005663,7F-09912.01.02,1.0,364,Nicaragua,10005,Caribbean & Central America,10018,LMICs,11,ODA Grants,1,10,110,C01,0.021326,NaN,NaN,0.021533,NaN,NaN,NaN,NaN,0.021326,NaN,NaN,0.021533,NaN,NaN,NaN,NaN,NaN,NaN,11,0.021,NaN,NaN,NaN,NI 44 SOCIEDAD CULTURA Y MEMORIA,NI 44 Sociedad Cultura y Memoria,16061,Culture and recreation,160,I.6. Other Social Infrastructure & Services,11001,Central Government,NaN,11000,NaN,2017-11-01,2018-08-31,NaN,NaN,1,0,2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2017-11-01,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,16061,NaN,NaN,NaN,NaN,NaN


filter the oecd-data by commitment date. For the first subset 
* a: only take data into account with a commitmentdate greater then 2015-12-31. 
* b: remove every entry that cannot fall into any specific incomegroup-class b/c the project is focused on a greater region.
* b.1: check how many projects are missing a IncomegroupName
* c: group the data by recipientname and incomegroup 
* d: count how many projects with a donorname are in each incomegroup-subgroup of a country
* e: reset the both indicies and set the index to the RecipientName
* f: display the aggregated data

In [6]:
# a:
df = oecddf[oecddf["CommitmentDate"] > datetime(year=2015,month=12,day=31)]
# b:
df =  df[df['IncomegroupName'] != "Part I unallocated by income"]
# b.1"
display("total Projects with IncomegroupName: %d" %(df['IncomegroupName'].count()))
display("total Projects: %d" %(df['DonorName'].count()))
# c: 
gdf = df[['RecipientName','IncomegroupName','DonorName']].groupby(
    ["RecipientName","IncomegroupName"])
# d:
one = gdf.agg({'DonorName': 'count'})
# e:
one = one.reset_index().set_index("RecipientName") 
# f:
one

'total Projects with IncomegroupName: 484527'

'total Projects: 484527'

,IncomegroupName,DonorName
RecipientName,,
Afghanistan,LDCs,6864
Albania,UMICs,2798
Algeria,UMICs,1851
Angola,LDCs,1999
Antigua and Barbuda,UMICs,251
Argentina,UMICs,2443
Armenia,LMICs,3064
Azerbaijan,UMICs,1999
Bangladesh,LDCs,8030


filter the oecd-data by commitment date. For the second subset

* a: only take data into account with a commitmentdate between 2000-12-31 and 2003-01-01
* everything else as above

In [7]:
# a:
df = oecddf[oecddf["CommitmentDate"] > datetime(year=2000,month=12,day=31)]
df = df[df["CommitmentDate"] < datetime(year=2003,month=1,day=1)]
# b:
df =  df[df['IncomegroupName'] != "Part I unallocated by income"]
# b.1"
display("total Projects with IncomegroupName: %d" %(df['IncomegroupName'].count()))
display("total Projects: %d" %(df['DonorName'].count()))
# c: 
gdf = df[['RecipientName','IncomegroupName','DonorName']].groupby(
    ["RecipientName","IncomegroupName"])
# d:
two = gdf.agg({'DonorName': 'count'})
# e:
two = two.reset_index().set_index("RecipientName") 
# f:
two

'total Projects with IncomegroupName: 134610'

'total Projects: 134610'

,IncomegroupName,DonorName
RecipientName,,
Afghanistan,LDCs,1455
Albania,UMICs,1319
Algeria,UMICs,699
Angola,LDCs,1210
Anguilla,MADCTs,36
Antigua and Barbuda,UMICs,30
Argentina,UMICs,788
Armenia,LMICs,731
Azerbaijan,UMICs,693


* a) dropping the counts from the two disjunced subsets since there are no difference in classifaction in the selected timeranges of the subsets; every country has only one incomegroup-classification
* b) rename 'IncomegroupName' of the two datasets to 'ig-name-2015-plus' and 'ig-name-2001-2002'
* c) add the feature / column of the second set (2001-2002) to the first for compare
* d) compare both columns and store the result in the new column 'match'
* e) display the data side by side


In [8]:
# a:
one.drop(columns=['DonorName'],inplace=True)
two.drop(columns=['DonorName'],inplace=True)
# b: 
one.rename(columns = {'IncomegroupName': 'ig-name-2015-plus'},inplace=True)
two.rename(columns = {'IncomegroupName': 'ig-name-2001-2002'},inplace=True)
# c:
one['ig-name-2001-2002'] = two['ig-name-2001-2002']
# d:
one['match'] = np.where(one['ig-name-2001-2002'] == one['ig-name-2015-plus'], 'True', 'False')
# e:
one

,ig-name-2015-plus,ig-name-2001-2002,match
RecipientName,,,
Afghanistan,LDCs,LDCs,True
Albania,UMICs,UMICs,True
Algeria,UMICs,UMICs,True
Angola,LDCs,LDCs,True
Antigua and Barbuda,UMICs,UMICs,True
Argentina,UMICs,UMICs,True
Armenia,LMICs,LMICs,True
Azerbaijan,UMICs,UMICs,True
Bangladesh,LDCs,LDCs,True


It can be seen, that the both selected timeframes have identical incomegroup classifications(*). This implies, that within 15 years none of the country changed its classification. This does not match with the historical classification provided by the worldbank at https://datatopics.worldbank.org/world-development-indicators/the-world-by-income-and-region.html .  B/C those changes are backpropagated in the dataset of the oecd and replace earlier classification, this can lead to a false assumption, that in the past mostly UMICs got grands since they evolved from LDC or LMIC to UMIC.

(*) south sudan did change b/c in 2002 it was not listed as a country, that received grants

Some noteable countries affected by this effect are:
* thailand: changed 2009 form LMIC to UMIC
* cambodia: changed 2015 from LDC to LMIC
* china and ecuador: changed 2010 from LMIC to UMIC
* iraq: changed 2012 from LMIC to UMIC